In [1]:
import sys
sys.path.insert(0, "../python")
from vizdoom import *
import tensorflow as tf
from helper import create_agent
%load_ext autoreload
%autoreload 2

In [2]:
import os

In [3]:
# Initializes DoomGame from config file
def initialize_vizdoom(config_file):
    game = DoomGame()
    game.load_config(config_file)
    game.init()
    return game  

In [4]:
agent_file_path = "../agents/ddqn_dm2015_1.json"
config_file_path = "../config/linear_track.cfg"
results_dir = "../tmp/tmp_results/"
action_set = "basic_four"
params_file_path = "../experiments/linear_track_1/trial_20/train_data/net_data/main_net/params/train_model-100"

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
tf.reset_default_graph()
game = initialize_vizdoom(config_file_path)
agent = create_agent(agent_file_path,
                     game=game, 
                     params_file=params_file_path,
                     action_set=action_set,
                     output_directory=results_dir,
                     train_mode=True)
agent.initialize_new_episode()
current_screen = game.get_state().screen_buffer
agent.update_state(current_screen)
print("Q: ", agent.network.get_q_values(agent.state))

INFO:tensorflow:Restoring parameters from ../experiments/linear_track_1/trial_20/train_data/net_data/main_net/params/train_model-100


InternalError: Dst tensor is not initialized.
	 [[Node: main_network/save/RestoreV2_19/_13 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_87_main_network/save/RestoreV2_19", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

In [14]:
os.environ["CUDA_VISIBLE_DEVICES"]=""
tf.reset_default_graph()
game = initialize_vizdoom(config_file_path)
agent = create_agent(agent_file_path,
                     game=game, 
                     params_file=params_file_path,
                     action_set=action_set,
                     output_directory=results_dir,
                     train_mode=True)
agent.initialize_new_episode()
current_screen = game.get_state().screen_buffer
agent.update_state(current_screen)
print("Q: ", agent.network.get_q_values(agent.state))

INFO:tensorflow:Restoring parameters from ../experiments/linear_track_1/trial_20/train_data/net_data/main_net/params/train_model-100
Q:  [[-21.1530571  -25.13306999 -21.8796978  -19.99013329]]


In [15]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
tf.reset_default_graph()
game = initialize_vizdoom(config_file_path)
agent = create_agent(agent_file_path,
                     game=game, 
                     params_file=params_file_path,
                     action_set=action_set,
                     output_directory=results_dir,
                     train_mode=True)
agent.initialize_new_episode()
current_screen = game.get_state().screen_buffer
agent.update_state(current_screen)
print("Q: ", agent.network.get_q_values(agent.state))

INFO:tensorflow:Restoring parameters from ../experiments/linear_track_1/trial_20/train_data/net_data/main_net/params/train_model-100
Q:  [[-21.1530571  -25.13306999 -21.8796978  -19.99013329]]
